In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader

In [2]:
transform= transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)

In [5]:
class CustomData1(Dataset):
    def __init__(self, annoted_file, root_dir, transform=None):
        self.annotations=pd.read_csv(annoted_file)
        self.root_dir=root_dir
        self.transform=transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        img_path= os.path.join( self.root_dir,self.annotations.iloc[index,0])
        
        image=Image.open(img_path)
        label= torch.tensor(int(self.annotations.iloc[index,1]))
        
        if self.transform:
            image=self.transform(image)
            
        return (image,label)

In [6]:
annoted_file="C:/Users/HP/Py Code/NN/Pytorch/Pytorch/data/dogs-vs-cats/cats_dogs.csv"
root_dir="C:/Users/HP/Py Code/NN/Pytorch/Pytorch/data/dogs-vs-cats/cats_dogs_resized"

In [8]:
dataset = CustomData1(annoted_file, root_dir, transform)

In [9]:
trainset, testset= torch.utils.data.random_split(dataset, [5,5])

In [10]:
trainloader = DataLoader(trainset, batch_size=1,
                                         shuffle=True)


In [11]:
testloader = DataLoader(testset, batch_size=1,
                                         shuffle=False)

In [12]:
model = torchvision.models.googlenet(weights="DEFAULT")


Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\googlenet-1378be20.pth
100%|██████████████████████████████████████████████████████████████████████████████| 49.7M/49.7M [00:56<00:00, 926kB/s]


In [13]:
for param in model.parameters():
    param.requires_grad = False

In [14]:
model.fc=nn.Linear(1024,2)

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001)

In [18]:
for epoch in range(10):
    total_loss=0.0
    batch=0
    for batch, (inputs,labels) in enumerate(trainloader):
    
        pred= model(inputs)
        loss=criterion(pred,labels)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item() #per epoch
        
    print(f'epoch: {epoch+1} loss:{total_loss/batch:.3f}')
            
    
print("Finished training of images (5 batches)")
    

epoch: 1 loss:0.591
epoch: 2 loss:0.753
epoch: 3 loss:0.674
epoch: 4 loss:0.638
epoch: 5 loss:0.696
epoch: 6 loss:0.645
epoch: 7 loss:0.638
epoch: 8 loss:0.637
epoch: 9 loss:0.704
epoch: 10 loss:0.643
Finished training of images (5 batches)


In [ ]:
correct=0.0
total=0.0

with torch.no_grad():
    for images, labels in testloader:
        outputs= model(images)
        _, pred= torch.max(outputs.data,1)
        total+= labels.size(0)
        correct += (pred == labels).sum().item()
    print(total)
    print(correct)
print(f'Accuracy of model (5 batches): {100 * correct // total} %')        